In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from spacy.lang.en import English
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.preprocess.en import BasicPreprocessor
from mylib import datageneration as dg

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
tokenizer = English().tokenizer

In [4]:
df = pd.read_parquet("input/part01.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            30 non-null     object
 1   prompt          30 non-null     object
 2   street_address  30 non-null     object
 3   student_name    30 non-null     object
 4   username        30 non-null     object
 5   personal_url    30 non-null     object
 6   model           30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB


In [5]:
k=30
df.sample(k).head(k)

,text,prompt,street_address,student_name,username,personal_url,model
8,"In the maritime industry and port management, the use of innovative tools and methodologies is crucial for the development and testing of new ideas, prototypes, and products. One such tool that has shown great promise in enhancing this process is the integration of a learning launch platform within the design thinking process. This approach enables teams to learn and iterate more efficiently, ultimately leading to the creation of better solutions.\n\nA real-world example of the benefits of integrating a learning launch tool into the design thinking process can be seen in an incident that occurred near Bishan Street 13 (Block 194). At a shopping mall frequently visited by Anita Dewi, a group of port managers were tasked with finding a solution to improve the unloading and loading processes at the nearby docks. By using a learning launch platform as part of their design thinking process, they were able to quickly test and iterate on various ideas, including the implementation of new software systems and equipment. Ultimately, their efforts resulted in the successful implementation of a new system that significantly improved efficiency at the docks.\n\nAnother example of the benefits of this approach comes from the experiences of Renzo The Atienza, a maritime industry professional who used a learning launch tool while working on a project related to container shipping. Through rapid experimentation and iteration made possible by the platform, Renzo and his team were able to identify problems early on and make adjustments before investing too much time and resources. This led to the successful implementation of a new container tracking system that reduced errors and increased efficiency for all parties involved.\n\nThe integration of a learning launch platform can also benefit companies and professionals through online communities and resources. For instance, the WhatsApp group whatsapp.com/-iam-raik-official-wernecke-official is dedicated to maritime industry and port management professionals looking to connect and share knowledge related to innovation and technology. By providing access to these valuable resources, the learning launch tool becomes even more powerful and effective in driving success and innovation within the industry.\n\nIn conclusion, the integration of a learning launch tool within the design thinking process can greatly enhance the development and testing of ideas, prototypes, and products in the maritime industry and port management. Real-world examples like the one near Bishan Street 13, the experience of Renzo The Atienza, and the active community at whatsapp.com/-iam","[INST] Write a short essay on the following topic:\n In Maritime Industry and Port Management, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Bishan Street 13 (Block 194), at a shopping mall frequented by Anita Dewi\n 2. Give an example that involved someone named renzo.the.atienza.8025\n 3. Give an example that involved whatsapp.com/-iam-raik-official-wernecke-official- [/INST]\n",Bishan Street 13 (Block 194),Anita Dewi,renzo.the.atienza.8025,whatsapp.com/-iam-raik-official-wernecke-official-,Mixtral-8x7B-Instruct-v0.1
22,"In the field of architecture and urban design, designers are constantly seeking innovative ways to generate creative ideas and insights during the design thinking process. One approach that has gained popularity is the integration of mind mapping tools. Mind mapping is a visual representation of information that allows for the organization and connection of concepts and ideas. By incorporating this method into the design thinking process, architects and urban designers can enhance their ability to think creatively and develop innova

In [6]:
pre = BasicPreprocessor()
rows = []
for t in tqdm(df.itertuples()):
    street_address = str(getattr(t, "street_address")),
    print(street_address)
    student_name = str(getattr(t, "student_name")),
    username = str(getattr(t, "username")),
    personal_url = str(getattr(t, "personal_url")),
    text = pre(getattr(t, "text"))
    sp_tokens = tokenizer(text)
    tokens = [token.text for token in sp_tokens]
    trailing_whitespace = [bool(token.whitespace_) for token in sp_tokens]
    rows.append({
        "document": f"rh_{uuid.uuid4().hex}",
        "full_text": text,
        "tokens": tokens,
        "trailing_whitespace": trailing_whitespace,
        "labels": dg.labels(
            text=text, 
            tokens=sp_tokens, 
            street_address=street_address,
            student_name=student_name,
            username=username,
            personal_url=personal_url,
        ),
        "street_address": street_address,
        "student_name": student_name,
        "username": username,
        "personal_url": personal_url,
    })


0it [00:00, ?it/s]

('Tampines Street 41 (Block 402)',)
v=('Tampines Street 41 (Block 402)',)


AttributeError: 'tuple' object has no attribute 'lower'

In [ ]:
%%time
with open("output/rh01.json", "w") as f:
    json.dump(rows, f)

In [ ]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")